In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder 

In [ ]:
dns = pd.read_csv('DoS_DNS.csv')
ldap = pd.read_csv('DoS_LDAP.csv')
mssql = pd.read_csv('DoS_MSSQL.csv')
netbios = pd.read_csv('DoS_NETBIOS.csv')
ntp = pd.read_csv('DoS_NTP.csv')
snmp = pd.read_csv('DoS_SNMP.csv')
ssdp = pd.read_csv('DoS_SSDP.csv')
syn = pd.read_csv('DoS_SYN.csv')
tftp = pd.read_csv('DoS_TFTP.csv')
udp = pd.read_csv('DoS_UDP.csv')
udplag = pd.read_csv('DoS_UDPLAG.csv')

In [ ]:
data = pd.concat([dns, ldap, mssql, netbios, ntp, snmp, ssdp, udp, syn, udplag], ignore_index = True)

In [ ]:
data.shape

In [ ]:
data[' Label']

0         DrDoS_DNS
1         DrDoS_DNS
2         DrDoS_DNS
3         DrDoS_DNS
4         DrDoS_DNS
            ...    
399995      UDP-lag
399996      UDP-lag
399997      UDP-lag
399998      UDP-lag
399999      UDP-lag
Name:  Label, Length: 400000, dtype: object

In [ ]:
# Drop Unnamed:0, Unnamed:0.1 columns 
data = data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [ ]:
data.columns 

In [ ]:
data_real = data.replace(np.inf, np.nan)

In [ ]:
data_real.isnull().sum().sum()

In [ ]:
data_df = data_real.dropna(axis=0)

In [ ]:
data_df.isnull().sum().sum()

In [ ]:
data_df

In [ ]:
# data_df.to_csv('data_final.csv', index = False)
# from google.colab import files
# files.download('data_final.csv')

In [ ]:
data_X = data_df.drop([' Label', 'SimillarHTTP'], axis = 1)

In [ ]:
data_X.columns 

In [ ]:
data_X.shape 

In [ ]:
data_y = data_df[' Label']

In [ ]:
data_y.shape 

In [ ]:
data_df.isnull().sum().sum()

In [ ]:
data_y.unique()

In [ ]:
data_X 

### Label Encoding for the Dataset 

In [ ]:
le = LabelEncoder()
data_y_trans = le.fit_transform(data_y)

le_fid = LabelEncoder()
le_fid.fit(data_X['Flow ID'])
data_X['Flow ID'] = le_fid.fit_transform(data_X['Flow ID'])
le_SIP = LabelEncoder()
le_SIP.fit(data_X[' Source IP'])
data_X[' Source IP'] = le_SIP.fit_transform(data_X[' Source IP'])
le_DIP = LabelEncoder()
le_DIP.fit(data_X[' Destination IP'])
data_X[' Destination IP'] = le_DIP.fit_transform(data_X[' Destination IP'])
le_timestamp = LabelEncoder()
le_timestamp.fit(data_X[' Timestamp'])
data_X[' Timestamp'] = le_timestamp.fit_transform(data_X[' Timestamp'])

## Feature Selection 

In [ ]:
from sklearn.feature_selection import chi2 
from sklearn.feature_selection import SelectKBest 
from sklearn.ensemble import ExtraTreesClassifier

#selecting 20 best features
# select_best= SelectKBest(chi2, k=20)
# X_feat_20 = select_best.fit_transform(data_X, data_y_trans)
# X_feat_20.shape

model = ExtraTreesClassifier(random_state=42)
#creates an Extra Trees classifier model with a random state of 42. 
#The random state is used to initialize the random number generator, which can help to improve the reproducibility of the results.
model.fit(data_X, data_y_trans)

ExtraTreesClassifier(random_state=42)

In [ ]:
model.feature_importances_
#
#The model.feature_importances_ attribute of an LSTM model is a NumPy array that stores the importance of each feature in the model. 
#The importance of a feature is a measure of how much the feature contributes to the model's predictions

In [ ]:
feature_importance_std = pd.Series(model.feature_importances_, index=data_X.columns)
feature_importance_std.nlargest(20).plot(kind='bar', title='Standardised Dataset Feature Selection using ExtraTreesClassifier')

In [ ]:
data_X.shape 

In [ ]:
data_new_20features_X = data_X[[' Timestamp', ' Source Port', ' Min Packet Length', ' Fwd Packet Length Min', 'Flow ID', ' Packet Length Mean', ' Fwd Packet Length Max', ' Average Packet Size', ' ACK Flag Count', ' Avg Fwd Segment Size', ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Max Packet Length', ' Protocol', 'Fwd Packets/s', ' Flow Packets/s', 'Total Length of Fwd Packets', ' Subflow Fwd Bytes', ' Destination Port', ' act_data_pkt_fwd']]

In [ ]:
data_new_20features_X

### Train Test Split Normal dataset 84 Features 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y_trans, test_size = 0.30, random_state = 42)

In [ ]:
X_train.shape 

(273260, 85)

In [ ]:
X_test.shape 

(117112, 85)

### Standardization of the 84 Feature Dataset 

In [ ]:

from sklearn.preprocessing import StandardScaler 
ss = StandardScaler()
X_train_std = ss.fit_transform(X_train)
X_test_std = ss.fit_transform(X_test)

# **Spliting dataset**

In [ ]:
from sklearn.model_selection import train_test_split
X_train_20, X_test_20, y_train_20, y_test_20 = train_test_split(data_new_20features_X, data_y_trans, test_size = 0.30, random_state = 42)

### Standardization of the Feature Dataset 

In [ ]:
from sklearn.preprocessing import StandardScaler 
ss_20 = StandardScaler()
X_train_std_20 = ss_20.fit_transform(X_train_20)
X_test_std_20 = ss_20.fit_transform(X_test_20)

In [ ]:
X_train_std_20.shape 

(401914, 20)

In [ ]:
y_train_20.shape

(401914,)

In [ ]:
X_test_std_20.shape 

(172250, 20)

In [ ]:
y_test_20.shape 

(172250,)

###  LSTM model

In [ ]:
import keras
import keras.utils
from keras import utils as np_utils
from keras.utils import to_categorical

y_train_lstm_20 = np.array(y_train_20)
y_test_lstm_20 = np.array(y_test_20)

y_train_onehot_lstm = to_categorical(y_train_lstm_20,13)
y_test_one_hot_lstm = to_categorical(y_test_lstm_20,13)

X_train_lstm_20 = np.array(X_train_std_20)
X_test_lstm_20 = np.array(X_test_std_20)

In [ ]:
X_test_std_20

In [ ]:
X_train_lstm_20.shape[0] 

401914

In [ ]:
X_train_lstm_reshape = np.reshape(X_train_std_20, (X_train_lstm_20.shape[0], 1,  X_train_lstm_20.shape[1]))
X_test_lstm_reshape = np.reshape(X_test_std_20, (X_test_lstm_20.shape[0], 1, X_test_lstm_20.shape[1]))

In [ ]:
from keras.models import Sequential
from tensorflow.python.keras.layers import Dense, LSTM
from tensorflow.python.keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,
    patience=20, 
    restore_best_weights=True,
)
batch_size = 256

# Initialize the network
model_LSTM = Sequential()
model_LSTM.add(LSTM(8,input_dim=20, return_sequences=True)) 
model_LSTM.add(Dropout(0.1))
model_LSTM.add(LSTM(8,input_dim=20, return_sequences=False))
model_LSTM.add(Dropout(0.1))
model_LSTM.add(Dense(13))
model_LSTM.add(Activation('softmax'))

In [ ]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto',
      restore_best_weights=True)

In [ ]:
model_LSTM.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model_LSTM.fit(X_train_lstm_reshape, y_train_onehot_lstm, validation_data=(X_test_lstm_reshape, y_test_one_hot_lstm),batch_size=batch_size, epochs=10,callbacks=[monitor])

Epoch 1/10
1068/1068 [==============================] - 13s 7ms/step - loss: 1.5530 - accuracy: 0.4715 - val_loss: 0.8833 - val_accuracy: 0.6331
Epoch 2/10
1068/1068 [==============================] - 7s 7ms/step - loss: 0.7598 - accuracy: 0.7247 - val_loss: 0.4905 - val_accuracy: 0.8787
Epoch 3/10
1068/1068 [==============================] - 8s 8ms/step - loss: 0.4301 - accuracy: 0.8717 - val_loss: 0.2024 - val_accuracy: 0.9640
Epoch 4/10
1068/1068 [==============================] - 7s 6ms/step - loss: 0.2963 - accuracy: 0.9054 - val_loss: 0.1409 - val_accuracy: 0.9710
Epoch 5/10
1068/1068 [==============================] - 7s 7ms/step - loss: 0.2470 - accuracy: 0.9188 - val_loss: 0.1101 - val_accuracy: 0.9754
Epoch 6/10
1068/1068 [==============================] - 6s 6ms/step - loss: 0.2181 - accuracy: 0.9253 - val_loss: 0.0950 - val_accuracy: 0.9766
Epoch 7/10
1068/1068 [==============================] - 7s 7ms/step - loss: 0.2002 - accuracy: 0.9304 - val_loss: 0.0860 - val_accuracy

In [ ]:
y_perd_lstm = model_LSTM.predict_classes(X_test_lstm_reshape)

/usr/local/lib/python3.10/dist-packages/tensorflow/python/keras/engine/sequential.py:454: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print("Classification Report for LSTM: \n", classification_report(le.inverse_transform(y_test_lstm_20), le.inverse_transform(y_perd_lstm)))

In [ ]:
lstm_conf_mat = confusion_matrix(y_test_lstm_20, y_perd_lstm)
print("LSTM Confusion: \n", lstm_conf_mat)

In [ ]:
acc_score_lstm = accuracy_score(y_test_lstm_20, y_perd_lstm)
print("Accuracy Score for MLP: \n", acc_score_lstm*100)

Accuracy Score for MLP: 
 97.6031491222078
